In [21]:
import argparse
import os
import warnings
import yaml
import sys
sys.path.append('/home/pasti/PycharmProjects/Robot_CLOD/')

import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.accelerators import find_usable_cuda_devices
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from nanodet.data.collate import naive_collate
from nanodet.data.dataset import build_dataset
from nanodet.evaluator import build_evaluator
from nanodet.trainer.task import TrainingTask
from nanodet.trainer.latent_dist_task import LatentDistTrainingTask
from torchvision.transforms import ToTensor, ToPILImage
from nanodet.util import (
    NanoDetLightningLogger,
    cfg,
    convert_old_model,
    env_utils,
    load_config,
    load_model_weight,
    mkdir,
)
class TemporalBufferDataset(Dataset):

    def __init__(self, dataset_n, buffer_size=300):
        self.buffer_size = buffer_size
        # Sort tirod frames by file name
        sorted_dataset = natsorted(dataset_n, key=lambda x: x['img_info']['file_name'])
        # Get the indices by linear interpolation
        indices = np.linspace(0, len(dataset_n) - 1, buffer_size, dtype=int)
        # Subsample the sorted dataset by keeping every n-th item
        self.buffer_dataset = [sorted_dataset[i] for i in indices]
        del sorted_dataset

    def __getitem__(self, index):
        return self.buffer_dataset[index]

    def __len__(self):
        return self.buffer_size
    
    def update_buffer(self, dataset_np1, task):
        n_tokeep_samples = int(self.buffer_size*(task-1)/task)
        indices = np.linspace(0, self.buffer_size -1, n_tokeep_samples, dtype=int)
        subset_buffer = [self.buffer_dataset[i] for i in indices]

        #Order new task dataset by file name
        dataset_np1 = natsorted(dataset_np1, key=lambda x: x['img_info']['file_name'])
        #Take a random subset of the new task dataset
        new_indices = np.linspace(0, len(dataset_np1) - 1, self.buffer_size - n_tokeep_samples, dtype=int)
        subset_np1 = [dataset_np1[i] for i in new_indices]
        #Concate the two subsets to form the new buffer
        self.buffer_dataset = torch.utils.data.ConcatDataset([subset_buffer, subset_np1])


load_config(cfg, '/home/pasti/PycharmProjects/Robot_CLOD/eclod/cfg/TiROD.yml')
logger = NanoDetLightningLogger('test')

train_dataset = build_dataset(cfg.data.train, "train")

buffer_dataset = TemporalBufferDataset(train_dataset)

for item in buffer_dataset:
    print(item['img_info']['file_name'])

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
['bag', 'bottle', 'cardboard box', 'chair', 'potted plant', 'traffic cone', 'trashcan', 'ball', 'broom', 'garden hose', 'bucket', 'bycicle', 'gardening tool']
d1_h_frame1.png
d1_h_frame3.png
d1_h_frame6.png
d1_h_frame9.png
d1_h_frame12.png
d1_h_frame16.png
d1_h_frame18.png
d1_h_frame22.png
d1_h_frame24.png
d1_h_frame28.png
d1_h_frame31.png
d1_h_frame33.png
d1_h_frame37.png
d1_h_frame39.png
d1_h_frame43.png
d1_h_frame46.png
d1_h_frame49.png
d1_h_frame52.png
d1_h_frame56.png
d1_h_frame58.png
d1_h_frame61.png
d1_h_frame64.png
d1_h_frame67.png
d1_h_frame71.png
d1_h_frame73.png
d1_h_frame77.png
d1_h_frame79.png
d1_h_frame83.png
d1_h_frame86.png
d1_h_frame88.png
d1_h_frame92.png
d1_h_frame94.png
d1_h_frame98.png
d1_h_frame101.png
d1_h_frame104.png
d1_h_frame107.png
d1_h_frame111.png
d1_h_frame113.png
d1_h_frame116.png
d1_h_frame119.png
d1_h_frame122.png
d1_h_frame126.png
d1_h_frame128.png
d1_h_frame132.png
d1_

In [22]:
load_config(cfg, '/home/pasti/PycharmProjects/Robot_CLOD/eclod/cfg/SIDtask2.yml')
train_dataset = build_dataset(cfg.data.train, "train")
for item in train_dataset:
    print(item['img_info']['file_name'])
    break

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
['bag', 'bottle', 'cardboard box', 'chair', 'potted plant', 'traffic cone', 'trashcan', 'ball', 'broom', 'garden hose', 'bucket', 'bycicle', 'gardening tool']
d1_l_frame1.png


In [23]:
buffer_dataset.update_buffer(train_dataset, 2)

In [24]:
for item in buffer_dataset:
    print(item['img_info']['file_name'])

d1_h_frame1.png
d1_h_frame6.png
d1_h_frame12.png
d1_h_frame18.png
d1_h_frame24.png
d1_h_frame31.png
d1_h_frame37.png
d1_h_frame43.png
d1_h_frame49.png
d1_h_frame56.png
d1_h_frame61.png
d1_h_frame67.png
d1_h_frame73.png
d1_h_frame79.png
d1_h_frame86.png
d1_h_frame92.png
d1_h_frame98.png
d1_h_frame104.png
d1_h_frame111.png
d1_h_frame116.png
d1_h_frame122.png
d1_h_frame128.png
d1_h_frame134.png
d1_h_frame141.png
d1_h_frame147.png
d1_h_frame153.png
d1_h_frame159.png
d1_h_frame166.png
d1_h_frame171.png
d1_h_frame177.png
d1_h_frame183.png
d1_h_frame189.png
d1_h_frame196.png
d1_h_frame202.png
d1_h_frame208.png
d1_h_frame214.png
d1_h_frame221.png
d1_h_frame226.png
d1_h_frame232.png
d1_h_frame238.png
d1_h_frame244.png
d1_h_frame251.png
d1_h_frame257.png
d1_h_frame263.png
d1_h_frame269.png
d1_h_frame276.png
d1_h_frame281.png
d1_h_frame287.png
d1_h_frame293.png
d1_h_frame299.png
d1_h_frame306.png
d1_h_frame312.png
d1_h_frame318.png
d1_h_frame324.png
d1_h_frame331.png
d1_h_frame336.png
d1_h_frame3

In [25]:
load_config(cfg, '/home/pasti/PycharmProjects/Robot_CLOD/eclod/cfg/SIDtask3.yml')
train_dataset = build_dataset(cfg.data.train, "train")
for item in train_dataset:
    print(item['img_info']['file_name'])
    break

buffer_dataset.update_buffer(train_dataset, 2)
for item in buffer_dataset:
    print(item['img_info']['file_name'])

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
['bag', 'bottle', 'cardboard box', 'chair', 'potted plant', 'traffic cone', 'trashcan', 'ball', 'broom', 'garden hose', 'bucket', 'bycicle', 'gardening tool']
d2_h_frame1.png
d1_h_frame1.png
d1_h_frame12.png
d1_h_frame24.png
d1_h_frame37.png
d1_h_frame49.png
d1_h_frame61.png
d1_h_frame73.png
d1_h_frame86.png
d1_h_frame98.png
d1_h_frame111.png
d1_h_frame122.png
d1_h_frame134.png
d1_h_frame147.png
d1_h_frame159.png
d1_h_frame171.png
d1_h_frame183.png
d1_h_frame196.png
d1_h_frame208.png
d1_h_frame221.png
d1_h_frame232.png
d1_h_frame244.png
d1_h_frame257.png
d1_h_frame269.png
d1_h_frame281.png
d1_h_frame293.png
d1_h_frame306.png
d1_h_frame318.png
d1_h_frame331.png
d1_h_frame342.png
d1_h_frame354.png
d1_h_frame367.png
d1_h_frame379.png
d1_h_frame391.png
d1_h_frame403.png
d1_h_frame416.png
d1_h_frame428.png
d1_h_frame441.png
d1_h_frame452.png
d1_h_frame464.png
d1_h_frame477.png
d1_h_frame489.png
d1_h_frame501.